# SQL DATA ANALYSIS TASK

# Context
Fenix is a company that is in renewable energy. To this end, it sells solar kits to populations who are in need. Customers have the option of paying for the kit in cash or on credit. If the customer chooses to buy on credit, he can choose between one year (360 days) or two years (720). He is then asked to pay an amount per day (daily_rate), this amount depending on the duration of the loan and the type of kit he has chosen and corresponds to a day of light. So every time he pays that amount, he has a day of light. After 24 hours the system hangs and it has to make another payment to take advantage of the light. He can decide to pay for more than one day of light. The day number is then the whole part of the amount / daily rate ratio. After this number of days, the system still locks. When a customer spends more than 30 days without light, he is considered inactive. Paid vs Expected (PvE) means the ratio between the total amount paid by the client and the amount he should have paid up to date. It allows to appreciate the recovery.

Let's consider the following tables

Loan_current_daitails

loan_id	|customer_id|daily_rate|days_elapsed|Days_til_lock_out|Total_paid
--------|-----------|----------|------------|-----------------|----------
loan_id_1|customer_id_1|daily_rate_1|days_elapsed_1|Days_til_lock_out_1|Total_paid_1
loan_id_2|customer_id_2|daily_rate_2|days_elapsed_2|Days_til_lock_out_2|Total_paid_2
.........|.............|............|..............|...................|............
loan_id_n|customer_id_n	|daily_rate_n|days_elapsed_n|Days_til_lock_out_n|Total_paid_n


Person_democraphic

customer_id	|country|	region|	profession
------------|-------|---------|-----------
customer_id_1|	Country_1	|Region_1	|profession_1
.............|	............|..........	|............
customer_id_j|	Country_j|	Region_j	|profession_j


Sales_details

loan_id	|Product_type	|Date_fullfiled	|loan_duration
--------|----------------|--------------|-------------
loan_id_1	|Product_type_1	|Date_fullfiled_1	|loan_duration_1
...|	...	|...	|...
loan_id_n|	Product_type_n|	Date_fullfiled_n|	loan_duration_n

### Tables description

* **Loan_id:** unique loan identifier

* **Customer_id:** unique customer identifier

* **Days_elapsed:** the number of days the client spends with us

* **Daily_rate:** expected amount from customer per day

* **Days_til_lock_out:** If positive, then corresponds to the number of days of light remaining before the system hangs. If negative, corresponds to the number of days since the system was blocked (the client has been without light since this number of days)

* **Total_paid:** total amount to be paid by the customer

* **Country:** Customer's country

* **Region:** customer department

* **Profession:** client's profession

* **Product_type:** the type of kit purchased by the customer

* **Loan_duration:** the duration of the customer's loan (360 or 720 days)

* **Date_fullfiled:** the date of the sale

### The datasets
The datasets exist in csv formats and can be downloaded and saved on disk

In [17]:
#!pip install ipython-sql
%load_ext sql
%sql sqlite:///testdb.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
%%sql
DROP TABLE IF EXISTS Loan_current_daitails;
DROP TABLE IF EXISTS Person_democraphic;
DROP TABLE IF EXISTS Sales_details;

 * sqlite:///testdb.sqlite
Done.
Done.
Done.


[]

In [82]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("testdb.sqlite")
customers = pd.read_csv('Loan_current_daitails.csv')
orders = pd.read_csv('Person_democraphic.csv')
salesman = pd.read_csv('Sales_details.csv')

customers.to_sql("Loan_current_daitails", conn, index=False, if_exists="replace")
orders.to_sql("Person_democraphic", conn, index=False, if_exists="replace")
salesman.to_sql("Sales_details", conn, index=False, if_exists="replace")

### Write a SQL code to determine the total number of loans on the one hand and the total number of customers on the other

In [55]:
%%sql
SELECT COUNT(*) loans FROM Loan_current_daitails;

 * sqlite:///testdb.sqlite
Done.


loans
5000


In [38]:
%%sql
SELECT COUNT(*) customers FROM Person_democraphic;

 * sqlite:///testdb.sqlite
Done.


customers
4979


### Write a SQL code to determine the total number of loans by type of kit

In [58]:
%%sql
SELECT product_type kit, COUNT(loan_id) Number_of_Loans FROM Sales_details GROUP BY product_type;

 * sqlite:///testdb.sqlite
Done.


kit,Number_of_Loans
Fenix 2,841
Fenix 3,974
Fenix Power 2+ Tondeuse,18
Fenix Power 4 + Woofer BJ,35
Fenix Power TV 19in FTA BJ,14
"Fenix Power TV 19in FTA Upgrade, BJ",28
Fenix Power TV 24in FTA BJ,634
"Fenix Power TV 24in FTA Upgrade, BJ",177
Fenix Radio +3,1791
Fenix Radio +5,422


### Write SQL code to determine the number of inactive loans

In [59]:
%%sql
SELECT COUNT(loan_id) Inactive_Loans FROM Loan_current_daitails WHERE days_til_lockout < -30

 * sqlite:///testdb.sqlite
Done.


Inactive_Loans
626


### Write a SQL code to determine the regions where the total amount of payments exceeds 1,000,000

In [69]:
%%sql
SELECT p.region, SUM(l.total_paid) Total_Payment FROM Person_democraphic p, Loan_current_daitails l WHERE p.customer_id = l.customer_id GROUP BY p.region HAVING Total_payment > 1000000;

 * sqlite:///testdb.sqlite
Done.


region,Total_Payment
Alibori,8559985
Atacora,7802685
Atlantique,51293780
Borgou,18947292
Collines,11904427
Couffo,4812070
Donga,9936982
Littoral,3980519
Mono,11621621
Oueme,30027194


### Write a SQL code to classify by region and in ascending order, the loans according to the total amount paid

In [72]:
%%sql
SELECT p.region, SUM(l.total_paid) Total_Payment FROM Person_democraphic p, Loan_current_daitails l WHERE p.customer_id = l.customer_id GROUP BY p.region ORDER BY Total_Payment ASC 

 * sqlite:///testdb.sqlite
Done.


region,Total_Payment
Littoral,3980519
Couffo,4812070
Atacora,7802685
Alibori,8559985
Zou,9540211
Plateau,9691206
Donga,9936982
Mono,11621621
Collines,11904427
Borgou,18947292



### Write a SQL code to determine the top 10 occupations by region with the highest total amount of payments

In [92]:
%%sql
SELECT p.region, p.Profession, SUM(l.total_paid) Total_Payment FROM Person_democraphic p, Loan_current_daitails l WHERE p.customer_id = l.customer_id GROUP BY p.Profession ORDER BY p.region, Total_Payment DESC;

 * sqlite:///testdb.sqlite
Done.


region,Profession,Total_Payment
Alibori,Doctor,17445994
Atlantique,Security agent,16937067
Borgou,Ingenieur,17855161
Borgou,Driver,17376164
Borgou,Statistian,16998896
Mono,Farmer,17507403
Oueme,Teacher,19394189
Oueme,Cmmercial,17496730
Oueme,Tehnician,17474037
Zou,Nurse,19632331


In [112]:
%%sql
--#Top 10 participants in Alibori
SELECT p.region, p.Profession, SUM(l.total_paid) Total_Payment FROM Person_democraphic p, Loan_current_daitails l WHERE p.customer_id = l.customer_id AND p.region = 'Alibori' GROUP BY p.Profession ORDER BY Total_Payment DESC LIMIT 10;

 * sqlite:///testdb.sqlite
Done.


region,Profession,Total_Payment
Alibori,Ingenieur,1065369
Alibori,Teacher,1025911
Alibori,Tehnician,979185
Alibori,Cmmercial,964132
Alibori,Statistian,953493
Alibori,Nurse,903046
Alibori,Doctor,825486
Alibori,Driver,767897
Alibori,Farmer,566846
Alibori,Security agent,508620


In [114]:
%%sql
--#Top 10 Occupations from Atacora
SELECT p.region, p.Profession, SUM(l.total_paid) Total_Payment FROM Person_democraphic p, Loan_current_daitails l WHERE p.customer_id = l.customer_id AND p.region = 'Atacora' GROUP BY p.Profession ORDER BY Total_Payment DESC LIMIT 10;

 * sqlite:///testdb.sqlite
Done.


region,Profession,Total_Payment
Atacora,Nurse,1240646
Atacora,Teacher,1198674
Atacora,Doctor,933223
Atacora,Security agent,854759
Atacora,Cmmercial,853389
Atacora,Tehnician,761814
Atacora,Statistian,635455
Atacora,Ingenieur,520886
Atacora,Farmer,475102
Atacora,Driver,328737


In [115]:
%%sql
--#Top 10 Occupations from Atlantique
SELECT p.region, p.Profession, SUM(l.total_paid) Total_Payment FROM Person_democraphic p, Loan_current_daitails l WHERE p.customer_id = l.customer_id AND p.region = 'Atlantique' GROUP BY p.Profession ORDER BY Total_Payment DESC LIMIT 10;

 * sqlite:///testdb.sqlite
Done.


region,Profession,Total_Payment
Atlantique,Teacher,6062410
Atlantique,Security agent,5587249
Atlantique,Driver,5457680
Atlantique,Nurse,5394546
Atlantique,Ingenieur,5302315
Atlantique,Farmer,5041556
Atlantique,Doctor,4916428
Atlantique,Tehnician,4790191
Atlantique,Cmmercial,4545867
Atlantique,Statistian,4195538
